In [187]:
# %pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
import copy
import time
# from dtaidistance import dtw

# Load files

In [215]:
file_test = '50words_TEST'
# file_test = 'X_test.txt'
file_train = '50words_TRAIN'

# Read files
with open(file_test) as f:
    ts_test = f.read().splitlines()
with open(file_train) as f:
    ts_train = f.read().splitlines()


# Train Data
ts_train = [x.split() for x in ts_train]
ts_train = [[float(s) for s in row] for row in ts_train] 
ts_train = np.array(ts_train) # convert to numpy array
train_classes = ts_train[:,0]  # get the classes
train_data = copy.deepcopy(ts_train)
ts_train = np.delete(ts_train, np.s_[:1], axis=1) # remove the first column, which is the class

# Test Data
ts_test = [x.split() for x in ts_test]
ts_test = [[float(s) for s in row] for row in ts_test]
ts_test = np.array(ts_test) # convert to numpy array
test_classes = ts_test[:,0] # get the classes
test_data = copy.deepcopy(ts_test)
ts_test = np.delete(ts_test, np.s_[:1], axis=1) # remove the first column, which is the class


# ts_test = pd.Series(ts_test)
# ts_train = pd.Series(ts_train)

In [216]:
# x = 
# y = ts_train[5]
# plt.plot(x)
# plt.plot(y)
# plt.show()

# Let's Warp!! 
# The DTW Implementation, from scratch.

In [279]:
def DTW(s1,s2,window_size,weight=(1,1,1)):
#     s1 = np.insert(ts1,0,0)
#     s2 = np.insert(ts2,0,0)
    n,m = len(s1) ,len(s2)
    w1,w2,w3 = weight[0],weight[1],weight[2]
    matrix= np.ones((n,m)) * np.inf
#     for i in range(n):
#         matrix[i][0] = np.inf
#     for i in range(m):
#         matrix[0][i] = np.inf
        
    matrix[0][0] = 0
    for i in range(1,n):
        for j in range(max(1, i-window_size),min(m, i+window_size)):
            cost = abs(s1[i]-s2[j])
            matrix[i][j] = min(matrix[i-1][j] + w1*cost,
                               matrix[i-1][j-1] + w2*cost,
                               matrix[i][j-1] + w3*cost)
    distance = matrix[n-1][m-1]
    return distance,matrix

# 1NN classification

In [289]:
def one_NN(train_data,classes,sample,window_size,weight=(1,1,1)):
    min_dist = inf
    label = -1
    window_size = int(len(sample)/2)
    for i in range(len(train_data)):
        dist,matrix = DTW(train_data[i],sample,window_size)
        if(dist<min_dist):
            min_dist = dist
            label = classes[i]
    return label

In [ ]:
start_time = time.time()
weight = (1,2,1)
print(one_NN(train_data,test_classes,test_data[4],weight))
print(time.time() - start_time)

In [282]:
test_classes[4]
# def my_custom_norm(x, y):
#     return (x * x) + (y * y)

# # dist, cost, acc, path = dtw(x, y, dist=my_custom_norm)
# dist, cost, acc, path = dtw(x, y, dist=lambda x, y: norm(x - y,ord = 1))

4.0

In [200]:
def dtw(x, y, dist):
    """
    Computes Dynamic Time Warping (DTW) of two sequences.
    :param array x: N1*M array
    :param array y: N2*M array
    :param func dist: distance used as cost measure
    Returns the minimum distance, the cost matrix, the accumulated cost matrix, and the wrap path.
    """
    assert len(x)
    assert len(y)
    r, c = len(x), len(y)
    D0 = zeros((r + 1, c + 1))
    D0[0, 1:] = inf
    D0[1:, 0] = inf
    D1 = D0[1:, 1:] # view
    for i in range(r):
        for j in range(c):
            D1[i, j] = dist(x[i], y[j])
    C = D1.copy()
    for i in range(r):
        for j in range(c):
            D1[i, j] += min(D0[i, j], D0[i, j+1], D0[i+1, j])
    if len(x)==1:
        path = zeros(len(y)), range(len(y))
    elif len(y) == 1:
        path = range(len(x)), zeros(len(x))
    else:
        path = _traceback(D0)
    return D1[-1, -1] / sum(D1.shape), C, D1, path

In [58]:
def fastdtw(x, y, dist):
    """
    Computes Dynamic Time Warping (DTW) of two sequences in a faster way.
    Instead of iterating through each element and calculating each distance,
    this uses the cdist function from scipy (https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html)
    :param array x: N1*M array
    :param array y: N2*M array
    :param string or func dist: distance parameter for cdist. When string is given, cdist uses optimized functions for the distance metrics.
    If a string is passed, the distance function can be 'braycurtis', 'canberra', 'chebyshev', 'cityblock', 'correlation', 'cosine', 'dice', 'euclidean', 'hamming', 'jaccard', 'kulsinski', 'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'wminkowski', 'yule'.
    Returns the minimum distance, the cost matrix, the accumulated cost matrix, and the wrap path.
    """
    assert len(x)
    assert len(y)
    if ndim(x)==1:
        x = x.reshape(-1,1)
    if ndim(y)==1:
        y = y.reshape(-1,1)
    r, c = len(x), len(y)
    D0 = zeros((r + 1, c + 1))
    D0[0, 1:] = inf
    D0[1:, 0] = inf
    D1 = D0[1:, 1:]
    D0[1:,1:] = cdist(x,y,dist)
    C = D1.copy()
    for i in range(r):
        for j in range(c):
            D1[i, j] += min(D0[i, j], D0[i, j+1], D0[i+1, j])
    if len(x)==1:
        path = zeros(len(y)), range(len(y))
    elif len(y) == 1:
        path = range(len(x)), zeros(len(x))
    else:
        path = _traceback(D0)
    return D1[-1, -1] / sum(D1.shape), C, D1, path

In [288]:
A = np.array([1,3,4,9,8,2,1,5,7,3])
B = np.array([1,6,2,3,0,9,4,3,6,3])
dist,n = DTW(A,B,6)
dist

15.0

In [61]:
# def _dtw_distance(self, ts_a, ts_b, d = lambda x,y: abs(x-y)):
#         """Returns the DTW similarity distance between two 2-D
#         timeseries numpy arrays.

#         Arguments
#         ---------
#         ts_a, ts_b : array of shape [n_samples, n_timepoints]
#             Two arrays containing n_samples of timeseries data
#             whose DTW distance between each sample of A and B
#             will be compared
        
#         d : DistanceMetric object (default = abs(x-y))
#             the distance measure used for A_i - B_j in the
#             DTW dynamic programming function
        
#         Returns
#         -------
#         DTW distance between A and B
#         """

#         # Create cost matrix via broadcasting with large int
#         ts_a, ts_b = np.array(ts_a), np.array(ts_b)
#         M, N = len(ts_a), len(ts_b)
#         cost = sys.maxint * np.ones((M, N))

#         # Initialize the first row and column
#         cost[0, 0] = d(ts_a[0], ts_b[0])
#         for i in xrange(1, M):
#             cost[i, 0] = cost[i-1, 0] + d(ts_a[i], ts_b[0])

#         for j in xrange(1, N):
#             cost[0, j] = cost[0, j-1] + d(ts_a[0], ts_b[j])

#         # Populate rest of cost matrix within window
#         for i in xrange(1, M):
#             for j in xrange(max(1, i - self.max_warping_window),
#                             min(N, i + self.max_warping_window)):
#                 choices = cost[i - 1, j - 1], cost[i, j-1], cost[i-1, j]
#                 cost[i, j] = min(choices) + d(ts_a[i], ts_b[j])

#         # Return DTW distance given window 
#         return cost[-1, -1]

In [62]:
# def _dist_matrix(self, x, y):
#         """Computes the M x N distance matrix between the training
#         dataset and testing dataset (y) using the DTW distance measure
        
#         Arguments
#         ---------
#         x : array of shape [n_samples, n_timepoints]
        
#         y : array of shape [n_samples, n_timepoints]
        
#         Returns
#         -------
#         Distance matrix between each item of x and y with
#             shape [training_n_samples, testing_n_samples]
#         """
        
#         # Compute the distance matrix        
#         dm_count = 0
        
#         # Compute condensed distance matrix (upper triangle) of pairwise dtw distances
#         # when x and y are the same array
#         if(np.array_equal(x, y)):
#             x_s = shape(x)
#             dm = np.zeros((x_s[0] * (x_s[0] - 1)) // 2, dtype=np.double)
            
#             p = ProgressBar(shape(dm)[0])
            
#             for i in xrange(0, x_s[0] - 1):
#                 for j in xrange(i + 1, x_s[0]):
#                     dm[dm_count] = self._dtw_distance(x[i, ::self.subsample_step],
#                                                       y[j, ::self.subsample_step])
                    
#                     dm_count += 1
#                     p.animate(dm_count)
            
#             # Convert to squareform
#             dm = squareform(dm)
#             return dm
        
#         # Compute full distance matrix of dtw distnces between x and y
#         else:
#             x_s = np.shape(x)
#             y_s = np.shape(y)
#             dm = np.zeros((x_s[0], y_s[0])) 
#             dm_size = x_s[0]*y_s[0]
            
#             p = ProgressBar(dm_size)
        
#             for i in xrange(0, x_s[0]):
#                 for j in xrange(0, y_s[0]):
#                     dm[i, j] = self._dtw_distance(x[i, ::self.subsample_step],
#                                                   y[j, ::self.subsample_step])
#                     # Update progress bar
#                     dm_count += 1
#                     p.animate(dm_count)
        
#             return dm

In [63]:
# def predict(self, x):
#         """Predict the class labels or probability estimates for 
#         the provided data

#         Arguments
#         ---------
#           x : array of shape [n_samples, n_timepoints]
#               Array containing the testing data set to be classified
          
#         Returns
#         -------
#           2 arrays representing:
#               (1) the predicted class labels 
#               (2) the knn label count probability
#         """
        
#         dm = self._dist_matrix(x, self.x)

#         # Identify the k nearest neighbors
#         knn_idx = dm.argsort()[:, :self.n_neighbors]

#         # Identify k nearest labels
#         knn_labels = self.l[knn_idx]
        
#         # Model Label
#         mode_data = mode(knn_labels, axis=1)
#         mode_label = mode_data[0]
#         mode_proba = mode_data[1]/self.n_neighbors

#         return mode_label.ravel(), mode_proba.ravel()

In [64]:
# a = np.array([1,2,3])
# b = np.array([2,3,4,5])

In [65]:
# 0.13358939348611112
# x

In [ ]:
 to 

In [154]:
c = np.array([1,2,3,4,5])
d = np.array([1,6,3,8,9])
confusion_matrix(c,d)
accuracy_score(c,d)

0.40000000000000002